# TimeSeriesDataset and TimeSeriesDataModule

> TimeSeries.

In [ ]:
#| default_exp time.series

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np, pandas as pd

import torch, torch.nn as nn, pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
#| export
from dataclasses import dataclass, field, KW_ONLY
from beartype.typing import Optional, Union, Iterable

from iza.static import TIME, SERIES
from iza.utils import Slice

In [ ]:
#| export
from littyping.core import (Device)

from litds.abc.dfdm.base import set_dataset, BaseDataFrameDataModule
from litds.abc.dfds.base import BaseDataFrameDataset
from litds.types import (
    SequenceWithLength, SequencesWithLengths
)
from litds.mocks.time import MockTimeSeries
from litds.time.mixs import TimeDatasetMixin

In [ ]:
#| eval: False
df = MockTimeSeries(set_index=True).df
df.head()

,time,feature_0,feature_1,feature_2
series,,,,
0,0,4,8,0
0,1,0,0,0
0,2,0,2,2
0,3,6,0,4
0,4,7,4,3


## TimeSeriesDataset

In [ ]:
#| export
@dataclass
class TimeSeriesDataset(TimeDatasetMixin, BaseDataFrameDataset):    
    # time_key: str = TIME
    series_key: str = SERIES
    pad: Optional[bool] = True
    batch_first: Optional[bool] = True
    
    def __post_init__(self):
        super().__post_init__()
        if self.df.index.name == self.series_key:
            self.df.reset_index(inplace=True)
            
        self.features = self.df.columns.drop([self.series_key, self.time_key], errors='ignore')
        self.series_ids = self.df[self.series_key].unique()
                
    def __len__(self):
        return len(self.series_ids)
    

    def is_series(self, idx: Union[int, str]) -> bool:
        return idx in self.df.set_index(self.series_key).index

    def getone(self, idx: int) -> SequenceWithLength:        
        if not self.is_series(idx):
            raise IndexError(f'Index {idx} is not a known series')
        
        cur_id = self.df.set_index(self.series_key).index.get_loc(idx)

        df_sub = self.df.loc[cur_id]
        # make sure series of length one is a DataFrame not a Series
        if isinstance(df_sub, pd.Series):
            df_sub = pd.DataFrame(df_sub).T
        
        # sort by time
        df_sub = df_sub.sort_values(self.time_key)

        time_seq = torch.FloatTensor(df_sub[self.features].values).float()
        time_len = time_seq.shape[0]
        return time_seq, time_len
    
    def getmany(self, idx: Iterable, pad: Optional[bool]=None) -> SequencesWithLengths:
        pad = getattr(self, 'pad', pad)
        data = [self.getone(i) for i in idx]
        seqs, lens = zip(*data)
        if pad:
            seqs = pad_sequence(seqs, batch_first=self.batch_first)
        lens = torch.tensor(lens)
        return seqs, lens

    def __getitem__(self, idx:int) -> Union[SequenceWithLength, SequencesWithLengths]:        
        # Handle slice
        if isinstance(idx, slice):
            idx = self.series_ids[idx]
        
        # Handle series
        if isinstance(idx, Iterable):
            return self.getmany(idx)
        
        # Handle single item
        time_seq, time_len = self.getone(idx)
        return time_seq, time_len
    
    def getall(self):
        return self.getmany(self.series_ids)
    

### Example

In [ ]:
#| eval: False
tsd = TimeSeriesDataset(df)

In [ ]:
#| eval: False
tsd[[0, 1]]

(tensor([[[4., 8., 0.],
          [0., 0., 0.],
          [0., 2., 2.],
          [7., 2., 1.],
          [6., 0., 4.],
          [7., 4., 3.],
          [1., 5., 5.],
          [0., 1., 4.],
          [1., 2., 2.],
          [7., 0., 5.]],
 
         [[8., 4., 1.],
          [7., 2., 1.],
          [0., 8., 2.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
 tensor([10,  3]))

## TimeSeriesDataModule

In [ ]:
#| export
@set_dataset(TimeSeriesDataset)
class TimeSeriesDataModule(BaseDataFrameDataModule):
    series_key: str = SERIES
    time_key: str = TIME
    pad: Optional[bool] = True
    batch_first: Optional[bool] = True
    batch_size: Optional[int] = 64
    
    _: KW_ONLY = field(default=None, init=False)
    include_time: Optional[bool] = False
    device: Optional[Device] = None

    def setup(self, stage: Optional[str]=None):
        pass

    def train_dataloader(self):
        ds = self.make_dataset()
        self.train_ds = ds
        return DataLoader(ds, batch_size=self.batch_size, collate_fn=self.collate_fn)

    def collate_fn(self, batch):
        seqs, lens = zip(*batch)
        seqs = pad_sequence(seqs, batch_first=True)
        lens = torch.tensor(lens)
        return seqs, lens

    def getall(self, pad: Optional[bool]=True) -> Union[SequenceWithLength, SequencesWithLengths]:
        pad = getattr(self, 'pad', pad)
        return self.ds.getall(pad=pad)

### Example

In [ ]:
#| eval: False
tsdm = TimeSeriesDataModule(df)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()